In [482]:
import os
import io
import collections
import re
import itertools

from IPython import display

import numpy as np
import pandas as pd
import xarray as xr

# Updating CLIRAD parameter values in the CESM

New CLIRAD parameters are often first obtained and tested offline before being used in the CESM.  Because offline CLIRAD is in Fortran 77 and the version in CESM is in Fortran 90, the syntax declaring the parameters are different.  

In this notebook, it is shown what parameters are updated, where they are found in the offline and in the CESM code.  Parameters contained in small arrays, or tables, can simply be copy and pasted into the CESM version, while for parameters in large arrays of many dimensions, the process needs to be automated.


## Longwave coefficients

In the CESM, all the required parameters are defined in the subroutine `setcoef_lw` which can be found in the file `radiation.F90`.  These are:

* `xkw(9)` --- absorption coefficient for the first k-distribution interval due to water vapour line absorption (Table 4). $[cm^2/g]$
* `xke(9)` --- absorption coefficient for the first k-distribution function due to water vapour continuum absorption (Table 9). $[cm^2/g]$
* `mw(9)` --- ratio between neighbouring absorption coefficients for water vapour line absorption (Table 4)
* `aw(9)` --- A coefficients for temperature scaling for water vapour (Eq. 4.2)
* `bw(9)` --- B coefficients for temperature scaling for water vapour (eq. 4.2)
* `pm(9)` --- pressure-scaling parameter for water vapour absorption (Eq. 4.1) and (Table 3)
* `fkw(6,9)` --- planck-weighted k-distribution function due to h2o line absorption (Table 4)
* `gkw(6,3)` --- planck-weighted k-distribution function due to h2o line absorption in the 3 sub-bands (800-720, 620-720, 540-620 /cm) of band 3 (Table 10)
* `h11(26,31)`, `h12(26,31)`, `h13(26,31)` --- h2o transmission functions for band 1, 26 pressure intervals, 31 h2o-amount intervals.
* `h21(26,31)`, `h22(26,31)`, `h23(26,31)` --- h2o transmission functions for band 2, 26 pressure intervals, 31 h2o-amount intervals
* `h81(26,31)`, `h82(26,31)`, `h83(26,31)` --- h2o transmission functions for band 8, 26 pressure intervals, 31 h2o-amount intervals
* `c1(26,30)`, `c2(26,30)`, `c3(26,30)` --- co2 transmission functions for band 3, 26 pressure intervals, 30 co2-amount intervals 
* `o1(26,21)`, `o2(26,21)`, `o3(26,21)` --- o3 transmission functions for band 5, 26 pressure intervals, 31 o3-amount intervals


In offline CLIRAD, the transmission functions for water, co2 and o3 are defined in their own respective files.  The remaining parameters are defined at the beginning of the subroutine `irrad` in the main program file.

In [459]:
# directory containing offline clirad source code (HITRAN 2012 update)
dir_h2012 = '/nuwa_cluster/home/jackyu/radiation/clirad/LW/lee_hitran2012_update'

In [484]:
# directory containing offline clirad source code (HITRAN 1996 update)
dir_h1996 = '/nuwa_cluster/home/jackyu/radiation/clirad/LW/src'

In [501]:
with open(os.path.join(dir_h2012, 'CLIRAD_new_25cm_re.f')) as file:
    c = file.read()

In [684]:
def get_data_statements(s=''):
    '''
    Grab strings of the values contained in `data` statements in F77.
    e.g. `data a /45, 23.3, -1e+9/`
    
    Parameters
    ----------
    s: string in which to look for data statements
    dict_data: dictionary where keys are names of data array and
               values are strings 
               containing that data array's elements
    '''
    regex_data = re.compile(r'''
    \n \s+          # space immediately after newline 
                    # to avoid commented out data
    data \s+ 
    (\w+)           # name of the data variable
    \s* / 
    ([\s\w,.*+-]+)  # values of the data variable)
    /
    ''', re.VERBOSE)

    dict_data = dict(regex_data.findall(c))
    return dict_data

In [695]:
def get_data_shape(s=''):
    '''
    Get the shapes of arrays declared with fixed shapes.
    e.g. `real a(5,6), b(9), c(7,9)`
    
    Parameters:
    s: string containing Fortran code to look for shapes
       of constant-shape arrays of real numbers
    dict_shapes: dictionary where keys are names of arrays
                 and values are the shapes of the arrays
    '''
    regex_shapes = re.compile(r'''
    (\w+)          # name of the array
    (\(
    (?: \d,*)+     # do not group individual dimensions of an array
    \))
    ''', re.VERBOSE)

    lines_reals = [line 
                   for line in c.split('\n') 
                   if line.strip().startswith(('real', 'integer'))]

    lists_shapes = [regex_shapes.findall(line) for line in lines_reals]
    lists_shapes = [l for l in lists_shapes if l]

    list_shapes = itertools.chain.from_iterable(lists_shapes)

    # reverse tuple of dims 'cos Python packs arrays along rows first
    dict_shapes = {name: 
                   tuple(int(n) 
                         for n 
                         in reversed(shape.strip(')(').split(',')))
                   for name, shape in list_shapes}
    return dict_shapes
    

In [687]:
def get_data_array(s=''):
    '''
    Process a piece of Fortran code and returns a function that
    will return numpy arrays of arrays defined by data statements
    in the Fortran code.
    
    Parameters
    ----------
    s: string containing Fortran code
    func: function that returns a numpy array of a fortran array 
          of some name defined by a data statement
    '''
    dict_data = get_data_statements(s=s)
    dict_shapes = get_data_shape(s=s)
    
    def func(varname):
        '''
        Returns numpy array of array named varname
        
        Parameters
        ----------
        varname: string of array's name
        data: numpy array containing values of array named varname
        '''
        lists_nums = [nums.strip().split()[-1].split('*')
                      for nums in dict_data[varname].split(',')]

        lists_nums = [int(l[0]) * [l[1]] if len(l) == 2 
                      else l 
                      for l in lists_nums]

        list_nums = itertools.chain.from_iterable(lists_nums)
        data = np.array([float(num) for num in list_nums])

        data.reshape(dict_shapes[varname])
        return data
    return func



In [696]:
data_array = get_data_array(s=c)

In [699]:
data_array(varname='mw')

array([  6.,   6.,   8.,   6.,   6.,   8.,   9.,   6.,  16.])

### Transmission functions 

Coefficients are originally declared in the offline code, which is in F77.  In order to use these in the global model, they need to be re-expressed in F90, before being inserted into the global model source code.

In [483]:
files_trans_h1996 = {'h2o': 'h2o_tran3.F',
                     'co2': 'co2_tran4.F',
                     'o3': 'o3_tran3.F'}

files_trans_h2012 = {'h2o': 'h2o.2012H_25',
                     'co2': 'co2.2012H_25',
                     'o3': 'o3.2012H_25'}

In [448]:
band1_trans = dict(h2o=[1, 2, 8], co2=[1], o3=[1])
band2_trans = dict(h2o=[1, 2, 3], co2=[1, 2, 3], o3=[1, 2, 3])
num_pressures_trans = dict(h2o=26, co2=26, o3=26)
num_amounts_trans = dict(h2o=31, co2=30, o3=21)

In [443]:
def load_gas_trans(gas='h2o', dir_trans='./', files_trans=None):
    '''
    Get gas transmission values from CLIRAD-LW offline model and
    store them in xarray.DataArray
    
    Parameters
    ----------
    gas: name of the gas: h2o, co2, or o3
    dir_trans: directory containing files containing transmissions
    files_trans: dictionary of names to file names containing transmissions
    '''
    with open(os.path.join(dir_trans, files_trans[gas]), 
              encoding='utf-8', mode='r') as file:
        content = file.read()
        
    lines_data = (l for l in content.split('\n') if '&' in l)
    str_data = ''.join(lines_data)
    str_data = str_data.replace('&', '').replace('/', '')\
               .replace(',', '')
    
    df = pd.read_csv(io.StringIO(str_data), sep=r'\s+', header=None)
    data = np.squeeze(df).values
    
    num_band1s = len(band1_trans[gas])
    num_band2s = len(band2_trans[gas])
    num_pressures = num_pressures_trans[gas]
    num_amounts = num_amounts_trans[gas]
    
    data_ibs = data.reshape(num_band1s, int(data.shape[0] / num_band1s))
    
    pressure = range(1, num_pressures + 1)
    amount = range(1, num_amounts + 1)

    list_da_band1s = [xr.DataArray(data_ib.reshape(num_pressures, 
                                                   num_band2s, 
                                                   num_amounts),
                                   dims=['pressure', 'band2', 'amount'],
                                   coords=[pressure, 
                                           band2_trans[gas], 
                                           amount]) 
                      for data_ib in data_ibs]

    da = xr.concat(list_da_band1s, dim='band1')
    da.coords['band1'] = ('band1', band1_trans[gas])
    return da

In [430]:
def str_iw(da=None, gas='h2o', band1=1, band2=1, pressure=1):
    '''
    Returns string for gas transmittance for some gas, band1, band2
    and pressure in F90 syntax.
    
    Parameters
    ----------
    da: xarray.DataArray containing transmittance for gas.
    gas: name of the gas corresponding to da.
    band1: band 1. Can be 1, 2, or 8 for h2o.  
           Can only be 1 for co2 and o3.
    band2: band 2.  Can be 1, 2, or 3.
    pressure: pressure interval.
    '''
    writedata = list(da.sel(band1=band1, 
                            band2=band2,
                            pressure=pressure).values)

    fmt_num = ' {:> 12e}_r8'

    if gas == 'h2o':
        str_gas = 'h{band1}{band2}'.format(band1=band1, band2=band2)
    elif gas == 'co2':
        str_gas = 'c{band2}'.format(band2=band2)
    elif gas == 'o3':
        str_gas = 'o{band2}'.format(band2=band2)
    
    numbers_lines = []
    while True:
        numbers = []
        try:
            for _ in range(5):
                numbers.append(writedata.pop(0))
        except IndexError:
                if numbers:
                    numbers_lines.append(numbers)
                break
        else:
            numbers_lines.append(numbers)


    str_lines = []        
    for nums in numbers_lines[:-1]:
        str_line = ','.join(fmt_num.format(num) for num in nums)
        str_line += ', &'
        str_lines.append(str_line)

    str_line = ','.join(fmt_num.format(num) for num in numbers_lines[-1])
    str_line += '/)'
    str_lines.append(str_line)

    str_line = '{} ({:2d}, :) = (/ &'.format(str_gas, pressure)
    str_lines.insert(0, str_line)    
    
    return '\n'.join(str_lines)

### CLIRAD-LW HITRAN '96 data

In [444]:
da_h2o = load_gas_trans(gas='h2o', dir_trans=dir_h1996, 
                        files_trans=files_trans_h1996)

In [445]:
da_co2 = load_gas_trans(gas='co2', dir_trans=dir_h1996, 
                        files_trans=files_trans_h1996)

In [446]:
da_o3 = load_gas_trans(gas='o3', dir_trans=dir_h1996, 
                       files_trans=files_trans_h1996)

In [440]:
#h2o
for band1 in da_h2o.coords['band1'].values:
    for pressure in da_h2o.coords['pressure'].values:
        for band2 in da_h2o.coords['band2'].values:
            print(str_iw(da=da_h2o, gas='h2o',
                         band1=band1, band2=band2,
                         pressure=pressure))

h11 ( 1, :) = (/ &
  9.999384e-01_r8,  9.999018e-01_r8,  9.998526e-01_r8,  9.997908e-01_r8,  9.997177e-01_r8, &
  9.996338e-01_r8,  9.995385e-01_r8,  9.994290e-01_r8,  9.993002e-01_r8,  9.991446e-01_r8, &
  9.989510e-01_r8,  9.987050e-01_r8,  9.983880e-01_r8,  9.979790e-01_r8,  9.974520e-01_r8, &
  9.967700e-01_r8,  9.958770e-01_r8,  9.946940e-01_r8,  9.931130e-01_r8,  9.909790e-01_r8, &
  9.880700e-01_r8,  9.841000e-01_r8,  9.786500e-01_r8,  9.711500e-01_r8,  9.608600e-01_r8, &
  9.468300e-01_r8,  9.277700e-01_r8,  9.020000e-01_r8,  8.674000e-01_r8,  8.217000e-01_r8, &
  7.627000e-01_r8/)
h12 ( 1, :) = (/ &
 -2.021000e-07_r8, -3.628000e-07_r8, -5.891000e-07_r8, -8.735000e-07_r8, -1.204000e-06_r8, &
 -1.579000e-06_r8, -2.002000e-06_r8, -2.494000e-06_r8, -3.093000e-06_r8, -3.852000e-06_r8, &
 -4.835000e-06_r8, -6.082000e-06_r8, -7.591000e-06_r8, -9.332000e-06_r8, -1.128000e-05_r8, &
 -1.347000e-05_r8, -1.596000e-05_r8, -1.890000e-05_r8, -2.241000e-05_r8, -2.672000e-05_r8, &
 -3.208000e-

In [433]:
# o3
for band1 in da_o3.coords['band1'].values:
    for pressure in da_o3.coords['pressure'].values:
        for band2 in da_o3.coords['band2'].values:
            print(str_iw(da=da_o3, gas='o3',
                         band1=band1, band2=band2,
                         pressure=pressure))

o1 ( 1, :) = (/ &
  9.999934e-01_r8,  9.999869e-01_r8,  9.999734e-01_r8,  9.999461e-01_r8,  9.998917e-01_r8, &
  9.997863e-01_r8,  9.995791e-01_r8,  9.991838e-01_r8,  9.984440e-01_r8,  9.971210e-01_r8, &
  9.948950e-01_r8,  9.914460e-01_r8,  9.865600e-01_r8,  9.800800e-01_r8,  9.716500e-01_r8, &
  9.604400e-01_r8,  9.452700e-01_r8,  9.246300e-01_r8,  8.971000e-01_r8,  8.618000e-01_r8, &
  8.180000e-01_r8/)
o2 ( 1, :) = (/ &
  6.531000e-11_r8,  5.926000e-11_r8, -1.646000e-10_r8, -1.454000e-09_r8, -7.376000e-09_r8, &
 -2.968000e-08_r8, -1.071000e-07_r8, -3.584000e-07_r8, -1.125000e-06_r8, -3.289000e-06_r8, &
 -8.760000e-06_r8, -2.070000e-05_r8, -4.259000e-05_r8, -7.691000e-05_r8, -1.264000e-04_r8, &
 -1.957000e-04_r8, -2.895000e-04_r8, -4.107000e-04_r8, -5.588000e-04_r8, -7.300000e-04_r8, &
 -9.199000e-04_r8/)
o3 ( 1, :) = (/ &
 -2.438000e-11_r8, -4.826000e-11_r8, -9.474000e-11_r8, -1.828000e-10_r8, -3.406000e-10_r8, &
 -6.223000e-10_r8, -1.008000e-09_r8, -1.412000e-09_r8, -1.244000e-09_

In [432]:
#co2
for band1 in da_co2.coords['band1'].values:
    for pressure in da_co2.coords['pressure'].values:
        for band2 in da_co2.coords['band2'].values:
            print(str_iw(da=da_co2, gas='co2',
                         band1=band1, band2=band2,
                         pressure=pressure))

c1 ( 1, :) = (/ &
  9.998565e-01_r8,  9.997643e-01_r8,  9.996389e-01_r8,  9.994803e-01_r8,  9.992765e-01_r8, &
  9.989960e-01_r8,  9.986000e-01_r8,  9.980480e-01_r8,  9.973220e-01_r8,  9.964040e-01_r8, &
  9.952640e-01_r8,  9.938430e-01_r8,  9.920500e-01_r8,  9.897900e-01_r8,  9.869500e-01_r8, &
  9.833500e-01_r8,  9.787900e-01_r8,  9.730700e-01_r8,  9.659300e-01_r8,  9.572200e-01_r8, &
  9.466000e-01_r8,  9.336600e-01_r8,  9.177800e-01_r8,  8.982000e-01_r8,  8.742000e-01_r8, &
  8.450000e-01_r8,  8.103000e-01_r8,  7.699000e-01_r8,  7.244000e-01_r8,  6.749000e-01_r8/)
c2 ( 1, :) = (/ &
 -1.841000e-07_r8, -4.666000e-07_r8, -1.050000e-06_r8, -2.069000e-06_r8, -3.601000e-06_r8, &
 -5.805000e-06_r8, -8.863000e-06_r8, -1.291000e-05_r8, -1.806000e-05_r8, -2.460000e-05_r8, &
 -3.317000e-05_r8, -4.452000e-05_r8, -5.944000e-05_r8, -7.884000e-05_r8, -1.036000e-04_r8, &
 -1.346000e-04_r8, -1.727000e-04_r8, -2.186000e-04_r8, -2.728000e-04_r8, -3.364000e-04_r8, &
 -4.102000e-04_r8, -4.948000e-04_r8

### CLIRAD-LW HITRAN 2012 data

In [449]:
da_h2o = load_gas_trans(gas='h2o', dir_trans=dir_h2012,
                        files_trans=files_trans_h2012)

In [450]:
da_co2 = load_gas_trans(gas='co2', dir_trans=dir_h2012,
                        files_trans=files_trans_h2012)

In [451]:
da_o3 = load_gas_trans(gas='o3', dir_trans=dir_h2012,
                       files_trans=files_trans_h2012)

In [452]:
#h2o
for band1 in da_h2o.coords['band1'].values:
    for pressure in da_h2o.coords['pressure'].values:
        for band2 in da_h2o.coords['band2'].values:
            print(str_iw(da=da_h2o, gas='h2o',
                         band1=band1, band2=band2,
                         pressure=pressure))

h11 ( 1, :) = (/ &
  9.999381e-01_r8,  9.999033e-01_r8,  9.998556e-01_r8,  9.997953e-01_r8,  9.997222e-01_r8, &
  9.996380e-01_r8,  9.995462e-01_r8,  9.994435e-01_r8,  9.993216e-01_r8,  9.991678e-01_r8, &
  9.989653e-01_r8,  9.986948e-01_r8,  9.983393e-01_r8,  9.978818e-01_r8,  9.972953e-01_r8, &
  9.965348e-01_r8,  9.955312e-01_r8,  9.941926e-01_r8,  9.924028e-01_r8,  9.900066e-01_r8, &
  9.867768e-01_r8,  9.823918e-01_r8,  9.764139e-01_r8,  9.682592e-01_r8,  9.571567e-01_r8, &
  9.420965e-01_r8,  9.217305e-01_r8,  8.942524e-01_r8,  8.574184e-01_r8,  8.088142e-01_r8, &
  7.462649e-01_r8/)
h12 ( 1, :) = (/ &
 -1.374000e-07_r8, -2.708000e-07_r8, -4.751000e-07_r8, -7.528000e-07_r8, -1.099000e-06_r8, &
 -1.483000e-06_r8, -1.891000e-06_r8, -2.361000e-06_r8, -2.959000e-06_r8, -3.751000e-06_r8, &
 -4.799000e-06_r8, -6.115000e-06_r8, -7.662000e-06_r8, -9.412000e-06_r8, -1.143000e-05_r8, &
 -1.378000e-05_r8, -1.658000e-05_r8, -1.990000e-05_r8, -2.375000e-05_r8, -2.818000e-05_r8, &
 -3.337000e-

In [454]:
#co2
for band1 in da_co2.coords['band1'].values:
    for pressure in da_co2.coords['pressure'].values:
        for band2 in da_co2.coords['band2'].values:
            print(str_iw(da=da_co2, gas='co2',
                         band1=band1, band2=band2,
                         pressure=pressure))

c1 ( 1, :) = (/ &
  9.998617e-01_r8,  9.997708e-01_r8,  9.996461e-01_r8,  9.994842e-01_r8,  9.992765e-01_r8, &
  9.989989e-01_r8,  9.986109e-01_r8,  9.980698e-01_r8,  9.973462e-01_r8,  9.964243e-01_r8, &
  9.952782e-01_r8,  9.938480e-01_r8,  9.920486e-01_r8,  9.897799e-01_r8,  9.869117e-01_r8, &
  9.832790e-01_r8,  9.786922e-01_r8,  9.729493e-01_r8,  9.658450e-01_r8,  9.571388e-01_r8, &
  9.465017e-01_r8,  9.334680e-01_r8,  9.173830e-01_r8,  8.974334e-01_r8,  8.727516e-01_r8, &
  8.425301e-01_r8,  8.061374e-01_r8,  7.633176e-01_r8,  7.144380e-01_r8,  6.607177e-01_r8/)
c2 ( 1, :) = (/ &
 -2.292000e-07_r8, -5.441000e-07_r8, -1.141000e-06_r8, -2.114000e-06_r8, -3.578000e-06_r8, &
 -5.719000e-06_r8, -8.775000e-06_r8, -1.292000e-05_r8, -1.827000e-05_r8, -2.504000e-05_r8, &
 -3.377000e-05_r8, -4.530000e-05_r8, -6.053000e-05_r8, -8.030000e-05_r8, -1.054000e-04_r8, &
 -1.368000e-04_r8, -1.751000e-04_r8, -2.213000e-04_r8, -2.765000e-04_r8, -3.421000e-04_r8, &
 -4.203000e-04_r8, -5.130000e-04_r8

In [453]:
# o3
for band1 in da_o3.coords['band1'].values:
    for pressure in da_o3.coords['pressure'].values:
        for band2 in da_o3.coords['band2'].values:
            print(str_iw(da=da_o3, gas='o3',
                         band1=band1, band2=band2,
                         pressure=pressure))

o1 ( 1, :) = (/ &
  9.999938e-01_r8,  9.999874e-01_r8,  9.999744e-01_r8,  9.999488e-01_r8,  9.998970e-01_r8, &
  9.997959e-01_r8,  9.995981e-01_r8,  9.992188e-01_r8,  9.985109e-01_r8,  9.972431e-01_r8, &
  9.951076e-01_r8,  9.917982e-01_r8,  9.871269e-01_r8,  9.810033e-01_r8,  9.731765e-01_r8, &
  9.630269e-01_r8,  9.495626e-01_r8,  9.315930e-01_r8,  9.079931e-01_r8,  8.779813e-01_r8, &
  8.412762e-01_r8/)
o2 ( 1, :) = (/ &
  2.533000e-09_r8,  4.172000e-09_r8,  4.619000e-09_r8,  1.937000e-09_r8, -5.367000e-09_r8, &
 -2.668000e-08_r8, -9.717000e-08_r8, -3.201000e-07_r8, -9.879000e-07_r8, -2.866000e-06_r8, &
 -7.657000e-06_r8, -1.834000e-05_r8, -3.846000e-05_r8, -7.061000e-05_r8, -1.170000e-04_r8, &
 -1.816000e-04_r8, -2.691000e-04_r8, -3.826000e-04_r8, -5.210000e-04_r8, -6.817000e-04_r8, &
 -8.626000e-04_r8/)
o3 ( 1, :) = (/ &
  7.983000e-12_r8,  1.064000e-11_r8,  3.991000e-11_r8,  2.395000e-11_r8,  1.543000e-10_r8, &
  3.273000e-10_r8,  7.557000e-10_r8,  1.943000e-09_r8,  4.760000e-09_

## Shortwave coefficients


CLIRAD-SW is split into the UV and IR spectral regions, which are handled by the subroutines `soluv` and `solir`, respectively.  These two subroutines both exist in the CESM version and the offline version.

In subroutine `solir`, the following parameters are defined:

* `xk` --- water vapour absorption coefficient for 10 k-intervals. [$cm^2/gm$] (Table 2)
* `hk` --- water vapour k-distribution function. [fraction] (Table 2)
* `ry` --- extinction coefficient for Rayleight scattering. [/mb] (Table 3)
* ...

In [481]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')